In [ ]:
!pip install itunes-app-scraper-dmi

In [ ]:
"""
App Store Scraper utility classes
"""
import json

class AppStoreUtils:
	"""
		Helper class to access the names of the other classes
	"""

	def get_entries(self, clazz_name):
		"""
			Get the members and their names from the function
			:param object clazz_name: the class object be called. 
			:returns object method_names: a JSON representation of the names.
		"""
		method_names  = {}
		for collection in dir(clazz_name):
			if not collection.startswith('__'):
				method_names[str(collection.replace('_', ' '))] = getattr(clazz_name, str(collection))
		return json.dumps({'names': method_names})

class AppStoreCollections:
	"""
	App store collection IDs
	Borrowed from https://github.com/facundoolano/app-store-scraper. These are
	the various collections displayed in the app store, usually on the front
	page.
	"""
	TOP_MAC = 'topmacapps'
	TOP_FREE_MAC = 'topfreemacapps'
	TOP_GROSSING_MAC = 'topgrossingmacapps'
	TOP_PAID_MAC = 'toppaidmacapps'
	NEW_IOS = 'newapplications'
	NEW_FREE_IOS = 'newfreeapplications'
	NEW_PAID_IOS = 'newpaidapplications'
	TOP_FREE_IOS = 'topfreeapplications'
	TOP_FREE_IPAD = 'topfreeipadapplications'
	TOP_GROSSING_IOS = 'topgrossingapplications'
	TOP_GROSSING_IPAD = 'topgrossingipadapplications'
	TOP_PAID_IOS = 'toppaidapplications'
	TOP_PAID_IPAD = 'toppaidipadapplications'

class AppStoreCategories:
	"""
	App Store category IDs
	Borrowed from https://apps.apple.com/us/genre/ios-medical/id6020. These are
	the app's categories.
	"""

	MEDICAL = 6020
	

class AppStoreMarkets:
	"""
	App Store store IDs per country
	Borrowed from https://github.com/facundoolano/app-store-scraper.
	"""
	
	



	US = 143441



class AppStoreException(Exception):
	"""
	Thrown when an error occurs in the App Store scraper
	"""
	pass

In [ ]:
"""
iTunes App Store Scraper
"""
import requests
import json
import time
import re
from datetime import datetime

from urllib.parse import quote_plus



class AppStoreScraper:
	"""
	iTunes App Store scraper
	This class implements methods to retrieve information about iTunes App
	Store apps in various ways. The methods are fairly straightforward. 
	"""

	def get_app_ids_for_query(self, term, num=100, page=1, country="us", lang="en"):
		"""
		Retrieve suggested app IDs for search query
		:param str term:  Search query
		:param int num:  Amount of items to return per page, default 50
		:param int page:  Amount of pages to return
		:param str country:  Two-letter country code of store to search in,
		                     default 'nl'
		:param str lang:  Language code to search with, default 'nl'
		:return list:  List of App IDs returned for search query
		"""
		if term is None or term == "":
			raise AppStoreException("No term was given")

		url = "https://search.itunes.apple.com/WebObjects/MZStore.woa/wa/search?clientApplication=Software&media=software&term="
		url += quote_plus(term)

		amount = int(num) * int(page)

		country = self.get_store_id_for_country(country)
		headers = {
			"X-Apple-Store-Front": "%s,24 t:native" % country,
			"Accept-Language": lang
		}

		try:
			result = requests.get(url, headers=headers).json()
		except ConnectionError as ce:
			raise AppStoreException("Cannot connect to store: {0}".format(str(ce)))
		except json.JSONDecodeError:
			raise AppStoreException("Could not parse app store response")

		return [app["id"] for app in result["bubbles"][0]["results"][:amount]]

	def get_app_ids_for_collection(self, collection="", category="", num=100, country="us", lang=""):
		"""
		Retrieve app IDs in given App Store collection
		Collections are e.g. 'top free iOS apps'.
		:param str collection:  Collection ID. One of the values in
		                        `AppStoreCollections`.
		:param int category:  Category ID. One of the values in
		                      AppStoreCategories. Can be left empty.
		:param int num:  Amount of results to return. Defaults to 50.
		:param str country:  Two-letter country code for the store to search in.
		                     Defaults to 'nl'.
		:param str lang: Dummy argument for compatibility. Unused.
		:return:  List of App IDs in collection.
		"""
		if not collection:
			collection = AppStoreCollections.TOP_FREE_IOS

		country = self.get_store_id_for_country(country)
		params = (collection, category, num, country)
		url = "http://ax.itunes.apple.com/WebObjects/MZStoreServices.woa/ws/RSS/%s/%s/limit=%s/json?s=%s" % params

		try:
			result = requests.get(url).json()
		except json.JSONDecodeError:
			raise AppStoreException("Could not parse app store response")

		return [entry["id"]["attributes"]["im:id"] for entry in result["feed"]["entry"]]

	def get_app_ids_for_developer(self, developer_id, country="us", lang=""):
		"""
		Retrieve App IDs linked to given developer
		:param int developer_id:  Developer ID
		:param str country:  Two-letter country code for the store to search in.
		                     Defaults to 'nl'.
		:param str lang: Dummy argument for compatibility. Unused.
		:return list:  List of App IDs linked to developer
		"""
		url = "https://itunes.apple.com/lookup?id=%s&country=%s&entity=software" % (developer_id, country)

		try:
			result = requests.get(url).json()
		except json.JSONDecodeError:
			raise AppStoreException("Could not parse app store response")

		if "results" in result:
			return [app["trackId"] for app in result["results"] if app["wrapperType"] == "software"]
		else:
			# probably an invalid developer ID
			return []

	def get_similar_app_ids_for_app(self, app_id, country="us", lang="en"):
		"""
		Retrieve list of App IDs of apps similar to given app
		This one is a bit special because the response is not JSON, but HTML.
		We extract a JSON blob from the HTML which contains the relevant App
		IDs.
		:param app_id:  App ID to find similar apps for
		:param str country:  Two-letter country code for the store to search in.
		                     Defaults to 'nl'.
		:param str lang:  Language code to search with, default 'nl'
		:return list:  List of similar app IDs
		"""
		url = "https://itunes.apple.com/us/app/app/id%s" % app_id

		country = self.get_store_id_for_country(country)
		headers = {
			"X-Apple-Store-Front": "%s,32" % country,
			"Accept-Language": lang
		}

		result = requests.get(url, headers=headers).text
		if "customersAlsoBoughtApps" not in result:
			return []

		blob = re.search(r"customersAlsoBoughtApps\":\s*(\[[^\]]+\])", result)
		if not blob:
			return []

		try:
			ids = json.loads(blob[1])
		except (json.JSONDecodeError, IndexError):
			return []

		return ids

	def get_app_details(self, app_id, country="us", lang="", flatten=True):
		"""
		Get app details for given app ID
		:param app_id:  App ID to retrieve details for. Can be either the
		                numerical trackID or the textual BundleID.
		:param str country:  Two-letter country code for the store to search in.
		                     Defaults to 'nl'.
		:param str lang: Dummy argument for compatibility. Unused.
		:param bool flatten: The App Store response may by multi-dimensional.
		                     This makes it hard to transform into e.g. a CSV,
		                     so if this parameter is True (its default) the
		                     response is flattened and any non-scalar values
		                     are removed from the response.
		:return dict:  App details, as returned by the app store. The result is
		               not processed any further, unless `flatten` is True
		"""
		try:
			app_id = int(app_id)
			id_field = "id"
		except ValueError:
			id_field = "bundleId"

		url = "https://itunes.apple.com/lookup?%s=%s&country=%s&entity=software" % (id_field, app_id, country)

		try:
			result = requests.get(url).json()
		except Exception:
			try:
				# handle the retry here. 
				# Take an extra sleep as back off and then retry the URL once. 
				time.sleep(2)
				result = requests.get(url).json()
			except Exception:
				raise AppStoreException("Could not parse app store response for ID %s" % app_id)

		try:
			app = result["results"][0]
		except IndexError:
			raise AppStoreException("No app found with ID %s" % app_id)

		# 'flatten' app response
		# responses are at most two-dimensional (array within array), so simply
		# join any such values
		if flatten:
			for field in app:
				if isinstance(app[field], list):
					app[field] = ",".join(app[field])

		return app

	def get_multiple_app_details(self, app_ids, country="us", lang=""):
		"""
		Get app details for a list of app IDs
		:param list app_id:  App IDs to retrieve details for
		:param str country:  Two-letter country code for the store to search in.
		                     Defaults to 'nl'.
		:param str lang: Dummy argument for compatibility. Unused.
		:return generator:  A list (via a generator) of app details
		"""
		for app_id in app_ids:
			try:
				time.sleep(1)
				yield self.get_app_details(app_id, country=country, lang=lang)
			except AppStoreException as ase:
				self._log_error(country, str(ase))
				continue

	def get_store_id_for_country(self, country):
		"""
		Get store ID for country code
		:param str country:  Two-letter country code
		:param str country:  Two-letter country code for the store to search in.
		                     Defaults to 'nl'.
		"""
		country = country.upper()

		if hasattr(AppStoreMarkets, country):
			return getattr(AppStoreMarkets, country)
		else:
			raise AppStoreException("Country code not found for {0}".format(country))

	def _log_error(self, app_store_country, message):
		"""
		Write the error to a local file to capture the error. 
		:param str app_store_country: the country for the app store
		:param str message: the error message to log
		"""
		app_log = "{0}_log.txt".format(app_store_country)
		errortime = datetime.now().strftime('%Y%m%d_%H:%M:%S - ')
		fh = open(app_log, "a")
		fh.write("%s %s \n" % (errortime,message))
		fh.close()

APP SCRAPING

**Diabete:M** - Extracting Similar application

In [ ]:

import pandas as pd  

scraper = AppStoreScraper()
results = scraper.get_app_ids_for_query("Diabete:M")
similar = scraper.get_similar_app_ids_for_app(results[0])

app_details = scraper.get_multiple_app_details(similar)

app_details_df = pd.DataFrame()
while True:
    try:
        itm = next(app_details)
    except StopIteration:
        break
    app_details_df = app_details_df.append(itm, ignore_index=True)

In [ ]:
app_details_df.to_csv('appDetails.csv')
import pandas as pd
similarity1 = pd.read_csv('appDetails.csv')
similarity1

,Unnamed: 0,advisories,appletvScreenshotUrls,artistId,artistName,artistViewUrl,artworkUrl100,artworkUrl512,artworkUrl60,averageUserRating,averageUserRatingForCurrentVersion,bundleId,contentAdvisoryRating,currency,currentVersionReleaseDate,description,features,fileSizeBytes,formattedPrice,genreIds,genres,ipadScreenshotUrls,isGameCenterEnabled,isVppDeviceBasedLicensingEnabled,kind,languageCodesISO2A,minimumOsVersion,price,primaryGenreId,primaryGenreName,releaseDate,releaseNotes,screenshotUrls,sellerName,sellerUrl,supportedDevices,trackCensoredName,trackContentRating,trackId,trackName,trackViewUrl,userRatingCount,userRatingCountForCurrentVersion,version,wrapperType
0,0,Frequent/Intense Medical/Treatment Information,NaN,1.200240e+09,Ambrosia Systems Inc.,https://apps.apple.com/us/developer/ambrosia-s...,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,3.45070,3.45070,com.ambrosiasys.LinkBluCon,17+,USD,2021-04-26T05:17:39Z,The LinkBluCon app can be used with BluCon (Tr...,NaN,55377920,Free,"6020,6013","Medical,Health & Fitness",NaN,0.0,1.0,software,"DA,NL,EN,FI,FR,DE,EL,HU,IT,JA,MK,PL,PT,RU,ZH,E...",13.0,0.0,6020.0,Medical,2017-05-15T08:51:45Z,Introduced Activity option which compares the ...,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,Ambrosia Systems Inc.,https://www.ambrosiasys.com,"iPhone5s-iPhone5s,iPadAir-iPadAir,iPadAirCellu...",LinkBluCon,17+,1.200240e+09,LinkBluCon,https://apps.apple.com/us/app/linkblucon/id120...,71.0,71.0,1.1.46,software
1,1,NaN,NaN,1.057764e+09,"Sugarmate, LLC",https://apps.apple.com/us/developer/sugarmate-...,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,4.82096,4.82096,com.wildvillage.sugartime,4+,USD,2020-09-21T01:36:55Z,Sugarmate is a companion app for users of the ...,iosUniversal,145726464,Free,"6020,6013","Medical,Health & Fitness",https://is2-ssl.mzstatic.com/image/thumb/Purpl...,0.0,1.0,software,"NL,EN,FR,IT,PT",10.3,0.0,6020.0,Medical,2016-10-12T20:24:17Z,- Fixes iOS 14 menu bug\n- Fixes multi-user lo...,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,"Sugarmate, LLC",https://sugarmate.io/,"iPhone5-iPhone5,iPadFourthGen-iPadFourthGen,iP...",Sugarmate,4+,1.111093e+09,Sugarmate,https://apps.apple.com/us/app/sugarmate/id1111...,7680.0,7680.0,2.6.23,software
2,2,NaN,NaN,7.950401e+08,"SocialDiabetes, SL",https://apps.apple.com/us/developer/socialdiab...,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,4.32609,4.32609,com.socialdiabetes.socialdiabetes,4+,USD,2021-05-01T19:22:03Z,SocialDiabetes helps you to control better you...,iosUniversal,411312128,Free,"6020,6013","Medical,Health & Fitness",https://is4-ssl.mzstatic.com/image/thumb/Purpl...,0.0,1.0,software,"CA,EN,FR,DE,IT,PT,ES",10.1,0.0,6020.0,Medical,2013-10-26T13:56:48Z,· Fixed a crash when getting CGM data\n· Some ...,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,"SocialDiabetes, SL",https://www.socialdiabetes.com/en,"iPhone5-iPhone5,iPadFourthGen-iPadFourthGen,iP...",SocialDiabetes - Diabetes app,4+,7.271184e+08,SocialDiabetes - Diabetes app,https://apps.apple.com/us/app/socialdiabetes-d...,46.0,46.0,4.11.10,software
3,3,Frequent/Intense Medical/Treatment Information,NaN,7.366122e+08,Pascal Freiburghaus,https://apps.apple.com/us/developer/pascal-fre...,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,4.60498,4.60498,com.lobotomo.diabetespal,17+,USD,2021-03-30T06:44:12Z,DiabetesPal is a diabetes management software....,NaN,29736960,Free,"6020,6013","Medical,Health & Fitness",NaN,0.0,1.0,software,"EN,DE",12.3,0.0,6020.0,Medical,2013-11-08T00:39:39Z,"This version fixes various crashes, that were ...",https://is5-ssl.mzstatic.com/imag

In [ ]:
import csv
  
# open input CSV file as source
# open output CSV file as result
with open("appDetails.csv", "r") as source:
    reader = csv.reader(source)
      
    with open("Diabetes:M.csv", "w") as result:
        writer = csv.writer(result)
        for r in reader:
            
            # Use CSV Index to remove a column from CSV
            
            writer.writerow((r[0], r[4], r[10], r[12],r[13],r[15],r[17],r[18],r[25],r[29],r[33],r[36],r[41]))

In [ ]:
import pandas as pd
similarity1 = pd.read_csv('Diabetes:M.csv')
similarity1

,Unnamed: 0,artistName,averageUserRatingForCurrentVersion,contentAdvisoryRating,currency,description,fileSizeBytes,formattedPrice,languageCodesISO2A,primaryGenreName,sellerName,trackCensoredName,userRatingCount
0,0,Ambrosia Systems Inc.,3.45070,17+,USD,The LinkBluCon app can be used with BluCon (Tr...,55377920,Free,"DA,NL,EN,FI,FR,DE,EL,HU,IT,JA,MK,PL,PT,RU,ZH,E...",Medical,Ambrosia Systems Inc.,LinkBluCon,71.0
1,1,"Sugarmate, LLC",4.82096,4+,USD,Sugarmate is a companion app for users of the ...,145726464,Free,"NL,EN,FR,IT,PT",Medical,"Sugarmate, LLC",Sugarmate,7680.0
2,2,"SocialDiabetes, SL",4.32609,4+,USD,SocialDiabetes helps you to control better you...,411312128,Free,"CA,EN,FR,DE,IT,PT,ES",Medical,"SocialDiabetes, SL",SocialDiabetes - Diabetes app,46.0
3,3,Pascal Freiburghaus,4.60498,17+,USD,DiabetesPal is a diabetes management software....,29736960,Free,"EN,DE",Medical,Pascal Freiburghaus,DiabetesPal,1205.0
4,4,SquareMed Software GmbH,4.69744,17+,USD,With DiabetesConnect you have your diabetes un...,66575360,Free,"EN,DE,ES",Medical,SquareMed Software GmbH,DiabetesConnect,195.0
5,5,etrada GmbH,4.53488,17+,USD,Disclaimer!\nDon't use this App for medical de...,34727936,Free,"EN,FI,DE",Medical,etrada GmbH,nightguard,43.0
6,6,ManageBGL Pty Ltd,3.50000,12+,USD,- No. 1 insulin dosing app in 11 countries - 3...,35214336,Free,EN,Medical,ManageBGL Pty Ltd,Jade Insulin Dose Calc,36.0
7,7,Cubesoft SARL,4.56078,4+,USD,See 4 days (7 on iPad) of your blood glucose t...,67051520,Free,"NL,EN,FR,DE,IT,PT,ZH,ES",Medical,CUBESOFT SARL,Diabetes Tracker - Diabeto Log,617.0
8,8,Tidepool Project,2.90244,12+,USD,Tidepool Mobile is a companion uploader and no...,36102144,Free,EN,Medical,Tidepool Project,Tidepool Mobile,41.0
9,9,Harald AI,4.53333,4+,USD,Happy Bob turns diabetes management into motiv...,47162368,Free,"EN,FI,FR,DE,PT,SV",Medical,Harald AI Oy,Happy Bob,75.0


**Diabetes Tracker**- Extracting Similar application

In [ ]:
import pandas as pd
scraper = AppStoreScraper()
results = scraper.get_app_ids_for_query("Diabetes Tracker")
similar = scraper.get_similar_app_ids_for_app(results[0])

app_details = scraper.get_multiple_app_details(similar)
app_details_df = pd.DataFrame()
while True:
    try:
        itm = next(app_details)
    except StopIteration:
        break
    app_details_df = app_details_df.append(itm, ignore_index=True)


In [ ]:
app_details_df.to_csv('appDetails1.csv')
import csv
  
# open input CSV file as source
# open output CSV file as result
with open("appDetails1.csv", "r") as source:
    reader = csv.reader(source)
      
    with open("Diabetes.csv", "w") as result:
        writer = csv.writer(result)
        for r in reader:
            
            # Use CSV Index to remove a column from CSV
            
            writer.writerow((r[0], r[4], r[10], r[12],r[13],r[15],r[17],r[18],r[25],r[29],r[33],r[36],r[41]))

In [ ]:
import pandas as pd
similarity2 = pd.read_csv('Diabetes.csv')
similarity2

,Unnamed: 0,artistName,averageUserRatingForCurrentVersion,contentAdvisoryRating,currency,description,fileSizeBytes,formattedPrice,languageCodesISO2A,primaryGenreName,sellerName,trackCensoredName,userRatingCount
0,0,Azumio Inc.,4.82822,4+,USD,RANKED #1 DIABETES APP FOR OVER 10 YEARS. \n- ...,238040064,Free,"AR,CA,CS,DA,NL,EN,FI,FR,DE,EL,HE,HU,ID,IT,JA,K...",Medical,Tom Xu,Glucose Buddy Diabetes Tracker,20370.0
1,1,LabStyle Innovation Ltd,4.89229,12+,USD,Wouldn't it be great if managing diabetes was ...,192622592,Free,"AR,MY,KM,CS,DA,NL,EN,ET,FI,FR,DE,HE,HI,ID,IT,J...",Medical,LabStyle Innovation Ltd,Dario Health,16442.0
2,2,Ascensia Diabetes Care US Inc.,3.15347,17+,USD,The new CONTOUR™DIABETES app syncs with a CONT...,88784896,Free,"HR,CS,DA,NL,EN,FI,FR,DE,EL,IT,NB,PL,PT,RO,RU,S...",Medical,Ascensia Diabetes Care US Inc.,CONTOUR DIABETES app (US),821.0
3,3,LifeScan Inc.,4.75481,17+,USD,The OneTouch Reveal™ mobile app has helped ove...,123582464,Free,"HR,CS,NL,EN,FR,DE,HU,IT,JA,PL,PT,SL,ES,VI",Medical,"LifeScan, Inc.",OneTouch Reveal,73555.0
4,4,mySugr GmbH,4.65087,4+,USD,"Get help with managing your diabetes, it's qui...",91116544,Free,"BG,CS,DA,NL,EN,ET,FI,FR,DE,EL,IT,LV,LT,NB,PL,P...",Medical,mySugr GmbH,mySugr - Diabetes Tracker Log,9054.0
5,5,Alisa Unzhakova,4.53232,4+,USD,The diabetes app is built for tracking blood s...,23273472,Free,EN,Medical,Alisa Unzhakova,Glucose tracker++,1454.0
6,6,Abbott Labs,3.00994,4+,USD,The FreeStyle LibreLink app allows you to moni...,49942528,Free,"AR,DA,NL,EN,FI,FR,DE,EL,HE,IT,JA,KO,NB,NN,PL,P...",Medical,Abbott Diabetes Care Inc,FreeStyle LibreLink - US,2013.0
7,7,Florin Uscatu,4.55923,4+,USD,This is a simple to use Diabetes logging app t...,32329728,Free,EN,Health & Fitness,Florin Uscatu,Diabetes Logs,1148.0
8,8,"Informed Data Systems, Inc.",4.50638,4+,USD,"One Drop is a free, award-winning app for peop...",305978368,Free,"AR,EN,FR,DE,IT,JA,PT,RU,ZH,ES,ZH",Health & Fitness,"Informed Data Systems, Inc",One Drop: Transform Your Life,18486.0
9,9,Walmart,1.72455,4+,USD,Our new ReliOn™ Life Patient Mobile App is a f...,16429056,Free,EN,Medical,Walmart,Walmart ReliOn Life,167.0


**Diasend** - Extracting Similar application

In [ ]:
import pandas as pd
scraper = AppStoreScraper()
results = scraper.get_app_ids_for_query("Diasend")
similar = scraper.get_similar_app_ids_for_app(results[0])

app_details = scraper.get_multiple_app_details(similar)
app_details_df = pd.DataFrame()
while True:
    try:
        itm = next(app_details)
    except StopIteration:
        break
    app_details_df = app_details_df.append(itm, ignore_index=True)

In [ ]:
app_details_df.to_csv('Glucose.csv')
import csv
  
# open input CSV file as source
# open output CSV file as result
with open("Glucose.csv", "r") as source:
    reader = csv.reader(source)
      
    with open("Glucose_list.csv", "w") as result:
        writer = csv.writer(result)
        for r in reader:
            
            # Use CSV Index to remove a column from CSV
            
            writer.writerow((r[0], r[4], r[10], r[12],r[13],r[15],r[17],r[18],r[25],r[29],r[33],r[36],r[41]))

In [ ]:
import pandas as pd
similarity3 = pd.read_csv('Glucose_list.csv')
similarity3

,Unnamed: 0,artistName,averageUserRatingForCurrentVersion,contentAdvisoryRating,currency,description,fileSizeBytes,formattedPrice,languageCodesISO2A,primaryGenreName,sellerName,trackCensoredName,userRatingCount
0,0,Together Against Diabetes,5.00000,12+,USD,Välkommen till T1D-appen! Här hittar du mängde...,20504576,Free,SV,Health & Fitness,Together Against Diabetes,T1D,1.0
1,1,Tandem Diabetes Care Inc.,3.47126,12+,USD,The t:simulator™ App is an easy way to explore...,238222336,Free,EN,Medical,"Tandem Diabetes Care, Inc.",t:simulator™ App,87.0
2,2,Rehne Martin Konsult & Ideutveckling AB,0.00000,4+,USD,Kolhydratskollen är en app för barn med diabet...,12137472,Free,EN,Medical,Rehne Martin Konsult & Ideutveckling AB,Kolhydratskollen,0.0
3,3,"Sugarmate, LLC",4.82096,4+,USD,Sugarmate is a companion app for users of the ...,145726464,Free,"NL,EN,FR,IT,PT",Medical,"Sugarmate, LLC",Sugarmate,7680.0
4,4,Venueve AB,0.00000,4+,USD,För barn och vuxna med typ 1-diabetes. Diabete...,66635776,Free,NaN,Health & Fitness,Venueve AB,DiabetesNinja,0.0
5,5,"Newyu, Inc",2.97368,17+,USD,LibreLinkUp allows you to monitor and support ...,88783872,Free,"AR,HR,CS,DA,NL,EN,FI,FR,DE,EL,HE,IT,JA,KO,NB,P...",Medical,"Newyu, Inc",LibreLinkUp,38.0
6,6,Dexcom,4.50350,4+,USD,Dexcom CLARITY is a diabetes management applic...,25407488,Free,"AR,BG,CS,DA,NL,EN,ET,FI,FR,DE,HE,HU,IT,JA,KO,L...",Medical,"DexCom, Inc.",Dexcom CLARITY,2856.0
7,7,Tidepool Project,2.90244,12+,USD,Tidepool Mobile is a companion uploader and no...,36102144,Free,EN,Medical,Tidepool Project,Tidepool Mobile,41.0
8,8,Cubesoft SARL,4.56078,4+,USD,See 4 days (7 on iPad) of your blood glucose t...,67051520,Free,"NL,EN,FR,DE,IT,PT,ZH,ES",Medical,CUBESOFT SARL,Diabetes Tracker - Diabeto Log,617.0
9,9,Ambrosia Systems Inc.,3.45070,17+,USD,The LinkBluCon app can be used with BluCon (Tr...,55377920,Free,"DA,NL,EN,FI,FR,DE,EL,HU,IT,JA,MK,PL,PT,RU,ZH,E...",Medical,Ambrosia Systems Inc.,LinkBluCon,71.0


**Blood Diary(Diabetes)** - Extracting Similar application

In [ ]:
import pandas as pd
scraper = AppStoreScraper()
results = scraper.get_app_ids_for_query("Blood Diary(Diabetes)")
similar = scraper.get_similar_app_ids_for_app(results[0])

app_details = scraper.get_multiple_app_details(similar)
app_details_df = pd.DataFrame()
while True:
    try:
        itm = next(app_details)
    except StopIteration:
        break
    app_details_df = app_details_df.append(itm, ignore_index=True)

In [ ]:

app_details_df.to_csv('Blood Diary(Diabetes).csv')
import csv
  
# open input CSV file as source
# open output CSV file as result
with open("Blood Diary(Diabetes).csv", "r") as source:
    reader = csv.reader(source)
      
    with open("Blood Diary.csv", "w") as result:
        writer = csv.writer(result)
        for r in reader:
            
            # Use CSV Index to remove a column from CSV
            
            writer.writerow((r[0], r[4], r[10], r[12],r[13],r[15],r[17],r[18],r[25],r[29],r[33],r[36],r[41]))

In [ ]:
import pandas as pd
similarity4 = pd.read_csv('Blood Diary.csv')
similarity4

,Unnamed: 0,artistName,averageUserRatingForCurrentVersion,contentAdvisoryRating,currency,description,fileSizeBytes,formattedPrice,languageCodesISO2A,primaryGenreName,sellerName,trackCensoredName,userRatingCount
0,0,Adappt LLC,4.69545,4+,USD,Glucose Tracker is built for tracking blood su...,47889408,Free,"EN,FR,DE,IT,PT,ZH,ES,ZH",Medical,Adappt LLC,Glucose - Blood Sugar Tracker,15380.0
1,1,"Informed Data Systems, Inc.",4.50638,4+,USD,"One Drop is a free, award-winning app for peop...",305978368,Free,"AR,EN,FR,DE,IT,JA,PT,RU,ZH,ES,ZH",Health & Fitness,"Informed Data Systems, Inc",One Drop: Transform Your Life,18486.0
2,2,mySugr GmbH,4.65087,4+,USD,"Get help with managing your diabetes, it's qui...",91116544,Free,"BG,CS,DA,NL,EN,ET,FI,FR,DE,EL,IT,LV,LT,NB,PL,P...",Medical,mySugr GmbH,mySugr - Diabetes Tracker Log,9054.0
3,3,Dexcom,4.50350,4+,USD,Dexcom CLARITY is a diabetes management applic...,25407488,Free,"AR,BG,CS,DA,NL,EN,ET,FI,FR,DE,HE,HU,IT,JA,KO,L...",Medical,"DexCom, Inc.",Dexcom CLARITY,2856.0
4,4,CalorieKing,3.53299,12+,USD,"Need a quick and easy way to check calories, c...",51964928,Free,EN,Health & Fitness,CalorieKing Wellness Solutions,CalorieKing Food Search,197.0
5,5,LifeScan Inc.,4.75481,17+,USD,The OneTouch Reveal™ mobile app has helped ove...,123582464,Free,"HR,CS,NL,EN,FR,DE,HU,IT,JA,PL,PT,SL,ES,VI",Medical,"LifeScan, Inc.",OneTouch Reveal,73555.0
6,6,Dexcom,2.97033,4+,USD,Only use this app if you have the Dexcom G6 or...,76788736,Free,EN,Medical,"DexCom, Inc.",Dexcom G6,3505.0
7,7,LabStyle Innovation Ltd,4.89229,12+,USD,Wouldn't it be great if managing diabetes was ...,192622592,Free,"AR,MY,KM,CS,DA,NL,EN,ET,FI,FR,DE,HE,HI,ID,IT,J...",Medical,LabStyle Innovation Ltd,Dario Health,16442.0
8,8,Abbott Labs,3.00994,4+,USD,The FreeStyle LibreLink app allows you to moni...,49942528,Free,"AR,DA,NL,EN,FI,FR,DE,EL,HE,IT,JA,KO,NB,NN,PL,P...",Medical,Abbott Diabetes Care Inc,FreeStyle LibreLink - US,2013.0
9,9,Dexcom,2.59057,4+,USD,The Dexcom Follow App is part of the Dexcom SH...,50295808,Free,EN,Medical,"DexCom, Inc.",Dexcom Follow,1209.0


**Intellin Diabetes Tracker Log** - Extracting Similar application

In [ ]:
import pandas as pd
scraper = AppStoreScraper()
results = scraper.get_app_ids_for_query("Intellin Diabetes Tracker Log")
similar = scraper.get_similar_app_ids_for_app(results[0])

app_details = scraper.get_multiple_app_details(similar)
app_details_df = pd.DataFrame()
while True:
    try:
        itm = next(app_details)
    except StopIteration:
        break
    app_details_df = app_details_df.append(itm, ignore_index=True)

In [ ]:
app_details_df.to_csv('Diabetes_Tracker_Log.csv')
import csv
  
# open input CSV file as source
# open output CSV file as result
with open("Diabetes_Tracker_Log.csv", "r") as source:
    reader = csv.reader(source)
      
    with open("Diabetes_Tracker.csv", "w") as result:
        writer = csv.writer(result)
        for r in reader:
            
            # Use CSV Index to remove a column from CSV
            
            writer.writerow((r[0], r[4], r[10], r[12],r[13],r[15],r[17],r[18],r[25],r[29],r[33],r[36],r[41]))

In [ ]:
import pandas as pd
similarity5 = pd.read_csv('Diabetes_Tracker.csv')
similarity5

,Unnamed: 0,artistName,averageUserRatingForCurrentVersion,contentAdvisoryRating,currency,description,fileSizeBytes,formattedPrice,languageCodesISO2A,primaryGenreName,sellerName,trackCensoredName,userRatingCount
0,0,fridayforward,4.20000,4+,USD,Diabetes Diary allows diabetics to record gluc...,12106752,Free,EN,Health & Fitness,Chris Bowley,Diabetes Diary,5.0
1,1,Quin Technology LTD,4.33333,17+,USD,Change the way you think about diabetes self m...,38853632,Free,EN,Medical,Quin Technology LTD,Quin: Diabetes Management,3.0
2,2,Alisa Unzhakova,4.53232,4+,USD,The diabetes app is built for tracking blood s...,23273472,Free,EN,Medical,Alisa Unzhakova,Glucose tracker++,1454.0
3,3,Sirma Medical Systems JSC,4.58376,17+,USD,"Designed for both smart phones and tablets, th...",102452224,Free,"BG,CA,NL,EN,DE,PT,RU,ES",Medical,Sirma Medical Systems AD,Diabetes:M,788.0
4,4,Cubesoft SARL,4.56078,4+,USD,See 4 days (7 on iPad) of your blood glucose t...,67051520,Free,"NL,EN,FR,DE,IT,PT,ZH,ES",Medical,CUBESOFT SARL,Diabetes Tracker - Diabeto Log,617.0
5,5,Pascal Freiburghaus,4.60498,17+,USD,DiabetesPal is a diabetes management software....,29736960,Free,"EN,DE",Medical,Pascal Freiburghaus,DiabetesPal,1205.0
6,6,"Health Preference, LLC",1.00000,12+,USD,* MANAGE your diabetes\n* TRACK your glucose l...,39186432,Free,"EN,FR,DE,IT,JA,KO,PT,RU,ZH,ES,ZH",Medical,"KHELS PREFERENS, OOO",Diabetes Tracker – Glucose Log,3.0
7,7,倩 赵,4.40921,4+,USD,Glucose Companion is a handy blood sugar and w...,15495168,Free,"EN,FR,DE,IT,JA,PT,RU,ZH,ES,TR",Medical,倩 赵,Glucose Companion,369.0
8,8,DiabTrend AI Analytics Korlatolt Felelossegu T...,4.65217,12+,USD,DiabTrend is a diabetes diary with a personali...,108710912,Free,EN,Health & Fitness,DiabTrend AI Analytics Korlatolt Felelossegu T...,DiabTrend - Diabetes Assistant,23.0
9,9,"SocialDiabetes, SL",4.32609,4+,USD,SocialDiabetes helps you to control better you...,411312128,Free,"CA,EN,FR,DE,IT,PT,ES",Medical,"SocialDiabetes, SL",SocialDiabetes - Diabetes app,46.0


Combining all the dataframe into one dataframe and removing duplicates rows.

In [ ]:
result = pd.concat( [similarity1, similarity2, similarity3, similarity4, similarity5], ignore_index=True)
result
# result = df.append([similarity1, similarity2, similarity3, similarity4, similarity5], ignore_index=True)
# result = result.drop(labels=['trackContentRating','userRatingCountForCurrentVersion'], axis=1)
# result

,Unnamed: 0,artistName,averageUserRatingForCurrentVersion,contentAdvisoryRating,currency,description,fileSizeBytes,formattedPrice,languageCodesISO2A,primaryGenreName,sellerName,trackCensoredName,userRatingCount
0,0,Ambrosia Systems Inc.,3.45070,17+,USD,The LinkBluCon app can be used with BluCon (Tr...,55377920,Free,"DA,NL,EN,FI,FR,DE,EL,HU,IT,JA,MK,PL,PT,RU,ZH,E...",Medical,Ambrosia Systems Inc.,LinkBluCon,71.0
1,1,"Sugarmate, LLC",4.82096,4+,USD,Sugarmate is a companion app for users of the ...,145726464,Free,"NL,EN,FR,IT,PT",Medical,"Sugarmate, LLC",Sugarmate,7680.0
2,2,"SocialDiabetes, SL",4.32609,4+,USD,SocialDiabetes helps you to control better you...,411312128,Free,"CA,EN,FR,DE,IT,PT,ES",Medical,"SocialDiabetes, SL",SocialDiabetes - Diabetes app,46.0
3,3,Pascal Freiburghaus,4.60498,17+,USD,DiabetesPal is a diabetes management software....,29736960,Free,"EN,DE",Medical,Pascal Freiburghaus,DiabetesPal,1205.0
4,4,SquareMed Software GmbH,4.69744,17+,USD,With DiabetesConnect you have your diabetes un...,66575360,Free,"EN,DE,ES",Medical,SquareMed Software GmbH,DiabetesConnect,195.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,40,"i-SENS,Inc.",3.71429,4+,USD,SmartLog: Smart blood glucose management app f...,23073792,Free,"HR,NL,EN,ET,FI,FR,DE,IT,KO,PL,PT,RU,ZH,ES,SV,VI",Health & Fitness,"i-SENS,Inc.",SmartLog for iPhone (i-SENS),7.0
203,41,Health Arx Technologies Private Limited,4.00000,12+,USD,Looking for a solution to manage & monitor you...,249282560,Free,EN,Health & Fitness,Health Arx Technologies Private Limited,BeatO Diabetes Management,38.0
204,42,Harald AI,4.53333,4+,USD,Happy Bob turns diabetes management into motiv...,47162368,Free,"EN,FI,FR,DE,PT,SV",Medical,Harald AI Oy,Happy Bob,75.0
205,43,Florin Uscatu,4.55923,4+,USD,This is a simple to use Diabetes logging app t...,32329728,Free,EN,Health & Fitness,Florin Uscatu,Diabetes Logs,1148.0


In [ ]:
result.drop_duplicates(subset ="artistName",
                     keep = 'first', inplace=True, ignore_index=True)
result.reset_index(drop=True)

,Unnamed: 0,artistName,averageUserRatingForCurrentVersion,contentAdvisoryRating,currency,description,fileSizeBytes,formattedPrice,languageCodesISO2A,primaryGenreName,sellerName,trackCensoredName,userRatingCount
0,0,Ambrosia Systems Inc.,3.45070,17+,USD,The LinkBluCon app can be used with BluCon (Tr...,55377920,Free,"DA,NL,EN,FI,FR,DE,EL,HU,IT,JA,MK,PL,PT,RU,ZH,E...",Medical,Ambrosia Systems Inc.,LinkBluCon,71.0
1,1,"Sugarmate, LLC",4.82096,4+,USD,Sugarmate is a companion app for users of the ...,145726464,Free,"NL,EN,FR,IT,PT",Medical,"Sugarmate, LLC",Sugarmate,7680.0
2,2,"SocialDiabetes, SL",4.32609,4+,USD,SocialDiabetes helps you to control better you...,411312128,Free,"CA,EN,FR,DE,IT,PT,ES",Medical,"SocialDiabetes, SL",SocialDiabetes - Diabetes app,46.0
3,3,Pascal Freiburghaus,4.60498,17+,USD,DiabetesPal is a diabetes management software....,29736960,Free,"EN,DE",Medical,Pascal Freiburghaus,DiabetesPal,1205.0
4,4,SquareMed Software GmbH,4.69744,17+,USD,With DiabetesConnect you have your diabetes un...,66575360,Free,"EN,DE,ES",Medical,SquareMed Software GmbH,DiabetesConnect,195.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,33,Chello Publishing,2.52941,4+,USD,Other calorie counting apps boast 5 MILLION+ F...,214206464,$4.99,EN,Health & Fitness,Chello Publishing Limited,Carbs & Cals: Diet & Diabetes,17.0
94,36,MyWay Digital Health,0.00000,12+,USD,My Diabetes * My Way (MDMW) is an online platf...,105210880,Free,NaN,Medical,MyWay Digital Health Limited,My Diabetes * My Way,0.0
95,39,Seong Eon Kim,4.00000,12+,USD,"Diary blood (blood pressure, blood sugar diary...",7944192,Free,"EN,KO",Health & Fitness,Seong Eon Kim,Blood Diary ( Diabetes ),3.0
96,40,"i-SENS,Inc.",3.71429,4+,USD,SmartLog: Smart blood glucose management app f...,23073792,Free,"HR,NL,EN,ET,FI,FR,DE,IT,KO,PL,PT,RU,ZH,ES,SV,VI",Health & Fitness,"i-SENS,Inc.",SmartLog for iPhone (i-SENS),7.0


In [ ]:
#Removes 0 values in the dataframe
drop = result[(result[['averageUserRatingForCurrentVersion','userRatingCount']] != 0).all(axis=1)]
#Removes the Nan values in the dataframe
drop = drop.dropna()
drop.reset_index(drop=True)

,Unnamed: 0,artistName,averageUserRatingForCurrentVersion,contentAdvisoryRating,currency,description,fileSizeBytes,formattedPrice,languageCodesISO2A,primaryGenreName,sellerName,trackCensoredName,userRatingCount
0,0,Ambrosia Systems Inc.,3.45070,17+,USD,The LinkBluCon app can be used with BluCon (Tr...,55377920,Free,"DA,NL,EN,FI,FR,DE,EL,HU,IT,JA,MK,PL,PT,RU,ZH,E...",Medical,Ambrosia Systems Inc.,LinkBluCon,71.0
1,1,"Sugarmate, LLC",4.82096,4+,USD,Sugarmate is a companion app for users of the ...,145726464,Free,"NL,EN,FR,IT,PT",Medical,"Sugarmate, LLC",Sugarmate,7680.0
2,2,"SocialDiabetes, SL",4.32609,4+,USD,SocialDiabetes helps you to control better you...,411312128,Free,"CA,EN,FR,DE,IT,PT,ES",Medical,"SocialDiabetes, SL",SocialDiabetes - Diabetes app,46.0
3,3,Pascal Freiburghaus,4.60498,17+,USD,DiabetesPal is a diabetes management software....,29736960,Free,"EN,DE",Medical,Pascal Freiburghaus,DiabetesPal,1205.0
4,4,SquareMed Software GmbH,4.69744,17+,USD,With DiabetesConnect you have your diabetes un...,66575360,Free,"EN,DE,ES",Medical,SquareMed Software GmbH,DiabetesConnect,195.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,31,Japps,2.00000,12+,USD,With this app you can easily track your blood ...,5567488,Free,"NL,EN",Medical,Japps,Blood Glucose Tracker,1.0
88,33,Chello Publishing,2.52941,4+,USD,Other calorie counting apps boast 5 MILLION+ F...,214206464,$4.99,EN,Health & Fitness,Chello Publishing Limited,Carbs & Cals: Diet & Diabetes,17.0
89,39,Seong Eon Kim,4.00000,12+,USD,"Diary blood (blood pressure, blood sugar diary...",7944192,Free,"EN,KO",Health & Fitness,Seong Eon Kim,Blood Diary ( Diabetes ),3.0
90,40,"i-SENS,Inc.",3.71429,4+,USD,SmartLog: Smart blood glucose management app f...,23073792,Free,"HR,NL,EN,ET,FI,FR,DE,IT,KO,PL,PT,RU,ZH,ES,SV,VI",Health & Fitness,"i-SENS,Inc.",SmartLog for iPhone (i-SENS),7.0


In [ ]:
#Removes 0 values in the dataframe
drop = result[(result[['userRatingCount']] >= 25).all(axis=1)]
#Removes the Nan values in the dataframe
drop = drop.dropna()
drop.reset_index(drop=True)

,Unnamed: 0,artistName,averageUserRatingForCurrentVersion,contentAdvisoryRating,currency,description,fileSizeBytes,formattedPrice,languageCodesISO2A,primaryGenreName,sellerName,trackCensoredName,userRatingCount
0,0,Ambrosia Systems Inc.,3.45070,17+,USD,The LinkBluCon app can be used with BluCon (Tr...,55377920,Free,"DA,NL,EN,FI,FR,DE,EL,HU,IT,JA,MK,PL,PT,RU,ZH,E...",Medical,Ambrosia Systems Inc.,LinkBluCon,71.0
1,1,"Sugarmate, LLC",4.82096,4+,USD,Sugarmate is a companion app for users of the ...,145726464,Free,"NL,EN,FR,IT,PT",Medical,"Sugarmate, LLC",Sugarmate,7680.0
2,2,"SocialDiabetes, SL",4.32609,4+,USD,SocialDiabetes helps you to control better you...,411312128,Free,"CA,EN,FR,DE,IT,PT,ES",Medical,"SocialDiabetes, SL",SocialDiabetes - Diabetes app,46.0
3,3,Pascal Freiburghaus,4.60498,17+,USD,DiabetesPal is a diabetes management software....,29736960,Free,"EN,DE",Medical,Pascal Freiburghaus,DiabetesPal,1205.0
4,4,SquareMed Software GmbH,4.69744,17+,USD,With DiabetesConnect you have your diabetes un...,66575360,Free,"EN,DE,ES",Medical,SquareMed Software GmbH,DiabetesConnect,195.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,47,"EKO GREIN, TOV",2.48193,4+,USD,"Just open the app, you can measure your heart ...",159925248,Free,"EN,FR,PT,ES",Health & Fitness,"EKO GREIN, TOV",Heart Beat. MyCalorieBalance,83.0
64,10,DIABNEXT,3.95238,12+,USD,DIABNEXT is a free app loaded with features to...,88270848,Free,"EN,FR,ES,ZH",Medical,DIABNEXT,DIABNEXT Manage your diabetes,84.0
65,23,Agnieszka Lisowska,4.74074,4+,USD,Tired of glucose trackers that you have to scr...,28185600,Free,EN,Medical,Agnieszka Lisowska,Doctor Sugar - Diabetes Type 2,108.0
66,28,Janki Panchani,4.60870,4+,USD,Blood glucose tracker by 9fit is an easy and c...,23181312,Free,EN,Health & Fitness,Janki Panchani,Blood Glucose Tracker & Diary,92.0


In [ ]:
#Sorting is based on averageUserRatingForCurrentVersion
sorting = drop.sort_values(["averageUserRatingForCurrentVersion"], axis=0, ascending=False)
sorting = sorting.reset_index(drop=True)
sorting

,Unnamed: 0,artistName,averageUserRatingForCurrentVersion,contentAdvisoryRating,currency,description,fileSizeBytes,formattedPrice,languageCodesISO2A,primaryGenreName,sellerName,trackCensoredName,userRatingCount
0,33,Fittur,4.99490,12+,USD,Does managing your diabetes seem overwhelming?...,41299968,Free,EN,Health & Fitness,Fittur LLC,Diabetes Food Tracker ~ Fittur,16653.0
1,40,LabStyle Innovation Ltd,4.89229,12+,USD,Wouldn't it be great if managing diabetes was ...,192622592,Free,"AR,MY,KM,CS,DA,NL,EN,ET,FI,FR,DE,HE,HI,ID,IT,J...",Medical,LabStyle Innovation Ltd,Dario Health,16442.0
2,37,"RxSaver, Inc.",4.84124,4+,USD,Save up to 85% on your prescriptions with RxSa...,10801152,Free,EN,Medical,"RxSaver, Inc.",RxSaver Prescription Discounts,19779.0
3,35,Azumio Inc.,4.82822,4+,USD,RANKED #1 DIABETES APP FOR OVER 10 YEARS. \n- ...,238040064,Free,"AR,CA,CS,DA,NL,EN,FI,FR,DE,EL,HE,HU,ID,IT,JA,K...",Medical,Tom Xu,Glucose Buddy Diabetes Tracker,20370.0
4,1,"Sugarmate, LLC",4.82096,4+,USD,Sugarmate is a companion app for users of the ...,145726464,Free,"NL,EN,FR,IT,PT",Medical,"Sugarmate, LLC",Sugarmate,7680.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,42,"Medtronic, Inc.",2.11628,4+,USD,Now there's a smarter way to manage your diabe...,43149312,Free,"AR,CS,DA,NL,EN,FI,FR,DE,HE,HU,IT,JA,KO,NB,PL,P...",Medical,"Medtronic, Inc.",Guardian Connect US,86.0
64,20,"Trividia Health, Inc.",2.06818,12+,USD,With TRUE MANAGER™ AIR Mobile App and TRUE MET...,25150464,Free,"EN,ES",Medical,"Trividia Health, Inc.",TRUE MANAGER™ AIR,220.0
65,16,"Roche Diabetes Care, Inc.",1.97823,12+,USD,The Accu-Chek Connect App puts diabetes manage...,111309824,Free,"HR,CS,DA,NL,EN,ET,FI,FR,DE,EL,IT,JA,KO,LV,LT,M...",Medical,"Roche Diabetes Care, Inc.",Accu-Chek® Connect App - US,643.0
66,9,Walmart,1.72455,4+,USD,Our new ReliOn™ Life Patient Mobile App is a f...,16429056,Free,EN,Medical,Walmart,Walmart ReliOn Life,167.0


In [ ]:
sorting.to_csv(r'/content/Final_apps_for_review.csv')

In [1]:
# data = sorting.drop([0,2,3,5,7,10,12,14,15,16,17,23,32,34,38,42,51,54,55,56,59,64,65,67], axis=0)
# data =data.reset_index(drop=True)
# data

In [2]:
# data.to_csv(r'/content/Final_apps_for_review.csv')

In [3]:
# #Creating the dataset
# import matplotlib.pyplot as plt
# plt.figure(figsize=(17,8))
# plt.bar(sort['trackCensoredName'],sort['averageUserRatingForCurrentVersion'],width = 0.4,alpha=0.9,color=['black'])
# #Adding the aesthetics
# plt.title('Top iOs Diabetes App Based on Rating')
# #xticks 
# plt.xticks(rotation=85,fontsize=10) 
# plt.xlabel('App Name')
# plt.ylabel('User Ratings') 
# #Show the plot
# plt.show()